In [21]:
import numpy as np
import pandas as pd
from cleantext import clean
import re 
import twint
import nest_asyncio

In [22]:
mapping = pd.read_csv('map.csv')
mapping.drop('Unnamed: 0', 1, inplace=True)
mapping_dict = mapping.to_dict()
mapping_dict = mapping_dict['emoji']
mapping_dict = {v: k for k, v in mapping_dict.items()}
mapping_dict

{'😜': 0, '😍': 1, '😉': 2, '🔥': 3, '💜': 4, '💯': 5}

In [23]:
nest_asyncio.apply()
emojis = "😜😍😉🔥💜💯"

for i, emoji in enumerate(emojis):
    print(emoji)
    config = twint.Config()
    config.Search = emoji
    config.Lang = "en"
    config.Limit = 500
    config.Store_json = True
    config.Output = f"{i+1}.json"
    config.Hide_output = True
    twint.run.Search(config)

😜
😍
😉
🔥
💜
💯


In [24]:
data = []

for i, emoji in enumerate(emojis):
    df = pd.read_json(f'{i+1}.json', lines=True)
    input = df.loc[:,'tweet' : 'language'].values
    input = pd.DataFrame(data = np.array(input), columns=["tweet", "language"])
    en = input[input['language'] == 'en']

    tweet = []
    for text in en['tweet'].values:
        text = clean(text, no_emoji=True)
        text = re.sub(r"http\S+", "", text)
        text = re.sub(r"@+", "", text)
        tweet.append(text)
    
    en['emoji'] = mapping_dict[emojis[i]]
    en['tweet'] = tweet
    en.drop('language', 1, inplace=True)

    data.extend(en.values)

c = pd.DataFrame(data = np.array(data), columns=["Tweet", "Emoji"])
c.to_csv("twitter_dataset.csv", index = False)
c

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Tweet,Emoji
0,"dianacoco_ pretty face, tiny body with big bre...",0
1,theapplehub that is creepy,0
2,ain't this a body-shaming!! it was meant to be...,0
3,"_calculator morning garth, trust you are well....",0
4,shivapunishbjp rahulgandhi dynastical bootlick...,0
...,...,...
828,stop fckin wimmie if you makin another mf feel...,5
829,y'all won that shit,5
830,please kindly retweet the thread of job vacanc...,5
831,jihoonisanaegi dm and get spoiled asap,5
